In [ ]:
### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [61]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [74]:
total_schools = school_data["School ID"].count()
total_students = student_data["Student ID"].count()
total_students_fmt = '{:,.0f}'.format(student_data["Student ID"].count())
total_budget = '${:,.2f}'.format(school_data["budget"].sum())
average_math_score = round(school_data_complete["math_score"].mean(),6)
average_reading_score = round(school_data_complete["reading_score"].mean(),6)
overall_passing_rate = round((average_math_score + average_reading_score)/2,6)
percent_passing_math = round((((school_data_complete["math_score"]>=70).sum())/total_students)*100,6)
percent_passing_reading = round((((school_data_complete["reading_score"]>=70).sum())/total_students)*100,6)
school_summary_df = pd.DataFrame({"Total Schools": [total_schools],
                           "Total Students": [total_students_fmt],
                           "Total Budget": [total_budget],
                           "Average Math Score": [average_math_score],
                           "Average Reading Score": [average_reading_score],
                           "% Passing Math": [percent_passing_math],
                           "% Passing Reading": [percent_passing_reading],
                           "% Overall Passing Rate": [overall_passing_rate]})
school_summary_df



,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [75]:
#top_performing_schools = school_data_complete.groupby(["school_name"])
#school_data_complete["percent_passing_rate"] = (school_data_complete["math_score"] + school_data_complete["reading_score"])/2
math_70 = student_data[student_data["math_score"] >= 70]
reading_70 = student_data[student_data["reading_score"] >= 70] 
math_70_gschool = math_70.groupby(["school_name"]).count()
reading_70_gschool = reading_70.groupby(["school_name"]).count()
student_data_gschool = student_data.groupby(["school_name"]).mean()
#school_data_gschool = school_data.groupby(["school_data",'type']).sum()


#school_gschool[["school_name","size","budget","per_student_budget","math_score_x","reading_score_x","per_pass_math"]]
school_gschool = pd.merge(math_70_gschool, reading_70_gschool, how="left", on=["school_name"])
school_gschool = pd.merge(school_gschool, student_data_gschool, how="left", on=["school_name"])
school_gschool = pd.merge(school_gschool, school_data, how="left", on=["school_name"])
school_gschool["per_student_budget"] = school_gschool["budget"] / school_gschool["size"]
school_gschool["per_pass_math"] = (school_gschool["math_score_x"] / school_gschool["size"]) * 100
school_gschool["per_pass_reading"] = (school_gschool["reading_score_y"] / school_gschool["size"]) * 100
school_gschool["overall_pass_rate"] = (school_gschool["per_pass_math"] + school_gschool["per_pass_reading"]) / 2

school_gschool = school_gschool.rename(columns={"school_name":""})
school_gschool.set_index("", inplace=True)

reduced_school_gschool = school_gschool[["type","size","budget","per_student_budget","math_score","reading_score", \
                                         "per_pass_math", "per_pass_reading", "overall_pass_rate"]]
reduced_school_gschool = reduced_school_gschool.rename(columns={"type":"School Type","size":"Total Students","budget":\
                                "Total School Budget","per_student_budget":"Per Student Budget","math_score":"Average Math Score",\
                                "reading_score": "Average Reading Score","per_pass_math":"% Passing Math","per_pass_reading":\
                                "% Passing Reading","overall_pass_rate":"% Overall Passing Rate"})

top_school_gschool = reduced_school_gschool.sort_values(["% Overall Passing Rate"], ascending=False)
top_school_gschool.head().style.format({"Total School Budget":"${:,.2f}","Per Student Budget":"${:,.2f}"})




,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.0619,83.9758,94.1335,97.0398,95.5867
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.4183,83.8489,93.2722,97.3089,95.2905
Pena High School,Charter,962,"$585,858.00",$609.00,83.8399,84.0447,94.5946,95.9459,95.2703
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.3515,83.8168,93.3924,97.139,95.2657
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.2742,83.9895,93.8677,96.5396,95.2037


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [76]:

bottom_school_gschool = reduced_school_gschool.sort_values(["% Overall Passing Rate"], ascending=True)
bottom_school_gschool.head().style.format({"Total School Budget":"${:,.2f}","Per Student Budget":"${:,.2f}"})
#####



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.8427,80.7447,66.3666,80.2201,73.2933
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.7118,81.158,65.9885,80.7392,73.3639
Huang High School,District,2917,"$1,910,635.00",$655.00,76.6294,81.1827,65.6839,81.3164,73.5002
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.0725,80.9664,66.0576,81.2224,73.64
Ford High School,District,2739,"$1,763,916.00",$644.00,77.1026,80.7463,68.3096,79.299,73.8043


## Math Scores by Grade

In [77]:
grades_math = student_data[["school_name","grade","math_score"]]
grades_math = grades_math.rename(columns={"school_name":""})
grades_math.set_index("", inplace=True)

grades_math = grades_math.pivot_table(index="", columns="grade",values="math_score")
grades_math = grades_math.rename_axis(None, axis=1)  
grades_math = grades_math[["9th","10th","11th","12th"]]
grades_math

,9th,10th,11th,12th
,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [78]:
grades_reading = student_data[["school_name","grade","reading_score"]]
grades_reading = grades_reading.rename(columns={"school_name":""})
grades_reading.set_index("", inplace=True)

grades_reading = grades_reading.pivot_table(index="", columns="grade",values="reading_score")
grades_reading = grades_reading.rename_axis(None, axis=1)  
grades_reading = grades_reading[["9th","10th","11th","12th"]]
grades_reading

,9th,10th,11th,12th
,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [79]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [80]:
reduced_school_gschool["Spending Ranges (Per Student)"] = pd.cut(reduced_school_gschool["Per Student Budget"],\
                                                                 spending_bins, labels=group_names)
group_spending_ranges = reduced_school_gschool.groupby("Spending Ranges (Per Student)")
group_spending_ranges = group_spending_ranges[["Average Math Score","Average Reading Score","% Passing Math",\
                        "% Passing Reading", "% Overall Passing Rate"]].mean()
group_spending_ranges

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [81]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [82]:
reduced_school_gschool["School Size"] = pd.cut(reduced_school_gschool["Total Students"],size_bins, labels=group_names)
group_school_size = reduced_school_gschool.groupby("School Size")
group_school_size = group_school_size[["Average Math Score","Average Reading Score","% Passing Math",\
                    "% Passing Reading", "% Overall Passing Rate"]].mean()
group_school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [84]:
group_school_type = reduced_school_gschool.groupby("School Type")
group_school_type = group_school_type[["Average Math Score","Average Reading Score","% Passing Math",\
                    "% Passing Reading", "% Overall Passing Rate"]].mean()
group_school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
